In [85]:
from collections import Counter


class Sequence: #name the sequence something you will remember, call it by name with extensions to spit results
    def __init__(self, filename): 
        self.filename = filename
        with open(str(filename), 'r') as f:
            self.h = f.readlines()[:1] #.h reads the header and saves it as a function of the class, in case we need this later
        with open(filename, 'r') as f:
            self.t = f.read()[(len(str(self.h)) - 6):] #.t is a string without the header
            self.sequence = ''.join(self.t).upper()
            self.sequence = self.sequence.replace("\n", "")
            self.sequence = self.sequence.replace("\r", "") #three lines of cleanup
        print(self.h)

    def nucleotide_count(self): #straightforward
        a = self.sequence.count('A')
        t = self.sequence.count('T')
        c = self.sequence.count('C')
        g = self.sequence.count('G')
        print("In the file {0}, there are: {1} Adenines, {2} Thymines, {3} Cytosines, and {4} Guanines.".format(self.filename,a,t,c,g))

    def ecori_digest_count(self): #also
        key = self.sequence.count('GAATTC')
        print("In the file, {0}, there are: {1} EcoRI Digestion Restricion Sites.".format(self.filename,key))

    def translate_to_proteins(self):
        start = 0
        self.proteins = []
        gencode = {
            'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M',
            'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
            'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K',
            'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R',
            'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',
            'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
            'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q',
            'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R',
            'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
            'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',
            'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E',
            'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G',
            'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',
            'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L',
            'TAC': 'Y', 'TAT': 'Y', 'TAA': '_', 'TAG': '_',
            'TGC': 'C', 'TGT': 'C', 'TGA': '_', 'TGG': 'W'}
        y = j = 0

        while y <= len(self.sequence): # y loop starts at zero and holds the last search start position
            if start == 0:
                start = self.sequence.find('ATG')
            else:
                start = self.sequence.find('ATG', y) #find an ATG
            protein = ''
            amino = ''
            protein_count = 0 #reset the counts
            for i in range(len(self.sequence[start::3])):# amino length 
                amino = self.sequence[i:i + 3] #amino length
                protein += gencode[amino] #make the protein one amino at a time
                protein_count += 1 #count the number, so we know where to search from
                if gencode[self.sequence[i:i + 3]] == '_': #stop codon
                    break #means stop
            self.proteins.append([start, protein]) #each one get's it's own index and list position
            y = start + 3 * (3 * protein_count - 2) #update search
            
            j += 1 #update protein count
        types = set(i[1] for i in self.proteins) #for printing only, could be removed, looked nice, fun practice
        print('The sequence {0}, \n contains {1} proteins of {2} types, including:\n {3}'.format(self.h,len(self.proteins), len(types), types))
        

    def translate_to_dna(self): #made this below using an easy iterator
        revgencode = {'I': ['ATA', 'ATC', 'ATT'],
                      'M': ['ATG'],
                      'T': ['ACA', 'ACC', 'ACG', 'ACT'],
                      'N': ['AAC', 'AAT'],
                      'K': ['AAA', 'AAG'],
                      'S': ['AGC', 'AGT', 'TCA', 'TCC', 'TCG', 'TCT'],
                      'R': ['AGA', 'AGG', 'CGA', 'CGC', 'CGG', 'CGT'],
                      'L': ['CTA', 'CTC', 'CTG', 'CTT', 'TTA', 'TTG'],
                      'P': ['CCA', 'CCC', 'CCG', 'CCT'],
                      'H': ['CAC', 'CAT'],
                      'Q': ['CAA', 'CAG'],
                      'V': ['GTA', 'GTC', 'GTG', 'GTT'],
                      'A': ['GCA', 'GCC', 'GCG', 'GCT'],
                      'D': ['GAC', 'GAT'],
                      'E': ['GAA', 'GAG'],
                      'G': ['GGA', 'GGC', 'GGG', 'GGT'],
                      'F': ['TTC', 'TTT'],
                      'Y': ['TAC', 'TAT'],
                      '_': ['TAA', 'TAG', 'TGA'],
                      'C': ['TGC', 'TGT'],
                      'W': ['TGG']}
        codon = []
        for p in str(self.sequence):
            pr = str(p) #this seemed to add the needed quotes, I know it looks redundant, help? 
            codon.append([revgencode[pr]])
        codon.append(revgencode['_']) #to force a stop codon into the list
        print(codon)




In [90]:
example = Sequence("Sequence.fasta.txt")
example.nucleotide_count()
example.ecori_digest_count()
example.translate_to_proteins()
print(list(i[0] for i in example.proteins))

['>gi|1271712971|emb|LT961839.1| unidentified phage genome assembly, chromosome: I\n']
In the file Sequence.fasta.txt, there are: 11118 Adenines, 10239 Thymines, 8106 Cytosines, and 9764 Guanines.
In the file, Sequence.fasta.txt, there are: 4 EcoRI Digestion Restricion Sites.
The sequence ['>gi|1271712971|emb|LT961839.1| unidentified phage genome assembly, chromosome: I\n'], 
 contains 434 proteins of 1 types, including:
 {'SLSHI_'}
[113, 178, 345, 395, 459, 865, 923, 977, 1025, 1091, 1156, 1268, 1321, 1429, 1560, 1614, 1662, 1747, 1854, 1913, 2103, 2241, 2356, 2463, 2562, 2619, 2761, 2857, 2914, 3045, 3161, 3220, 3286, 3390, 3552, 3613, 3733, 3820, 3882, 3986, 4040, 4164, 4295, 4433, 4559, 4629, 4687, 4782, 4885, 5022, 5076, 5124, 5238, 5310, 5369, 5425, 5479, 5527, 5584, 5664, 5712, 5787, 5868, 5924, 5993, 6053, 6113, 6313, 6517, 6605, 6661, 6713, 6778, 6917, 6968, 7185, 7242, 7305, 7395, 7495, 7557, 7615, 7680, 7732, 7929, 7978, 8053, 8132, 8208, 8427, 8559, 8634, 8859, 8908, 9082, 

In [78]:
protein_example = Sequence('protein.fasta')
print(protein_example.sequence)
protein_example.translate_to_dna()

['> protein sequence\t\n']
MKTYGNP
[[['ATG']], [['AAA', 'AAG']], [['ACA', 'ACC', 'ACG', 'ACT']], [['TAC', 'TAT']], [['GGA', 'GGC', 'GGG', 'GGT']], [['AAC', 'AAT']], [['CCA', 'CCC', 'CCG', 'CCT']], ['TAA', 'TAG', 'TGA']]


In [79]:
def dictinvert(d):
    inv = {}
    for k, v in d.items():
        keys = inv.setdefault(v, [])
        keys.append(k)
    return inv

dictinvert({
            'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M',
            'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
            'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K',
            'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R',
            'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',
            'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
            'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q',
            'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R',
            'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
            'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',
            'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E',
            'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G',
            'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',
            'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L',
            'TAC': 'Y', 'TAT': 'Y', 'TAA': '_', 'TAG': '_',
            'TGC': 'C', 'TGT': 'C', 'TGA': '_', 'TGG': 'W'})

{'I': ['ATA', 'ATC', 'ATT'],
 'M': ['ATG'],
 'T': ['ACA', 'ACC', 'ACG', 'ACT'],
 'N': ['AAC', 'AAT'],
 'K': ['AAA', 'AAG'],
 'S': ['AGC', 'AGT', 'TCA', 'TCC', 'TCG', 'TCT'],
 'R': ['AGA', 'AGG', 'CGA', 'CGC', 'CGG', 'CGT'],
 'L': ['CTA', 'CTC', 'CTG', 'CTT', 'TTA', 'TTG'],
 'P': ['CCA', 'CCC', 'CCG', 'CCT'],
 'H': ['CAC', 'CAT'],
 'Q': ['CAA', 'CAG'],
 'V': ['GTA', 'GTC', 'GTG', 'GTT'],
 'A': ['GCA', 'GCC', 'GCG', 'GCT'],
 'D': ['GAC', 'GAT'],
 'E': ['GAA', 'GAG'],
 'G': ['GGA', 'GGC', 'GGG', 'GGT'],
 'F': ['TTC', 'TTT'],
 'Y': ['TAC', 'TAT'],
 '_': ['TAA', 'TAG', 'TGA'],
 'C': ['TGC', 'TGT'],
 'W': ['TGG']}